In [1]:
!ls ../Data

atm.csv		 mosque.csv		      shopping_mall.csv
bank.csv	 pharmacy.csv		      SubCity-Coordinates.json
bus_station.csv  Population_per_subcity.json  SubCity-Populations.json
church.csv	 restaurants.csv	      supermarket.csv
gas_station.csv  school.csv		      train_station.csv
hospital.csv	 scripts


In [1]:
import pandas as pd

In [26]:
nearby_places_csvs = {
    'atms' : '../Data/atm.csv',
    'banks' : '../Data/bank.csv',
    'bus_stations' : '../Data/bus_station.csv',
    'churches' : '../Data/church.csv',
    'gas_stations' : '../Data/gas_station.csv',
    'hospitals' : '../Data/hospital.csv',
    'mosques' : '../Data/mosque.csv',
    'pharmacies' : '../Data/pharmacy.csv',
    'restaurants' : '../Data/restaurants.csv',
    'schools' : '../Data/school.csv',
    'train_stations' : '../Data/train_station.csv',
    'competitors_supermarket' : '../Data/supermarket.csv',
    'competitors_shopping_mall' : '../Data/shopping_mall.csv'
}
shopping_malls_csv = '../Data/shopping_mall.csv'
supermarkets_csv = '../Data/supermarket.csv'

In [27]:
nearby_places = pd.DataFrame()

for key in nearby_places_csvs:
    df = pd.read_csv(nearby_places_csvs[key])
    if 'permanently_closed' in df.columns:
        df = df[df['permanently_closed']!=True]
    df = df[['business_status', 'name', 'latitude', 'longitude']]
    df['type'] = key
    nearby_places = nearby_places.append(df)

supermarkets = pd.read_csv(supermarkets_csv)
supermarkets = supermarkets.append(pd.read_csv(shopping_malls_csv))
# supermarkets = supermarkets[['business_status', 'name', 'latitude', 'longitude', 'rating']]
supermarkets.head()

,Unnamed: 0,business_status,name,place_id,rating,latitude,longitude,rv_park,beauty_salon,aquarium,...,movie_rental,doctor,transit_station,bowling_alley,clothing_store,painter,hair_care,physiotherapist,hardware_store,permanently_closed
0,1472,OPERATIONAL,"AllMart Plus, Ayat Neighborhood, Addis Ababa",ChIJ-3rbDsqRSxYRKX0Zr55zXwY,NaN,9.020171,38.875108,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,1361,OPERATIONAL,Nahuta SuperMarket,ChIJ-4OScHqFSxYRPmsu7L3m4LA,5.0,9.022502,38.785852,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,1796,OPERATIONAL,Afran Mini Market,ChIJ-5L2YgKJSxYRj__fg6gaEy0,NaN,9.057363,38.724341,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,844,OPERATIONAL,251 Store ET Gerji store,ChIJ-60ObDCFSxYR8qG94H3GuPg,NaN,9.001777,38.810458,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1045,OPERATIONAL,Selam Super Market,ChIJ-7iRqQ6FSxYRDfOlAYW38c8,NaN,9.011199,38.796551,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [28]:
supermarkets['rating'].fillna('-', inplace=True)

In [29]:
for key in nearby_places_csvs:
    supermarkets[key] = 0

In [34]:
supermarkets.head()

,Unnamed: 0,business_status,name,place_id,rating,latitude,longitude,rv_park,beauty_salon,aquarium,...,churches,gas_stations,hospitals,mosques,pharmacies,restaurants,schools,train_stations,competitors_supermarket,competitors_shopping_mall
0,1472,OPERATIONAL,"AllMart Plus, Ayat Neighborhood, Addis Ababa",ChIJ-3rbDsqRSxYRKX0Zr55zXwY,-,9.020171,38.875108,0,0,0,...,7,3,3,0,2,28,4,0,4,6
1,1361,OPERATIONAL,Nahuta SuperMarket,ChIJ-4OScHqFSxYRPmsu7L3m4LA,5.0,9.022502,38.785852,0,0,0,...,15,1,18,4,9,105,28,0,19,21
2,1796,OPERATIONAL,Afran Mini Market,ChIJ-5L2YgKJSxYRj__fg6gaEy0,-,9.057363,38.724341,0,0,0,...,9,2,2,9,6,54,20,0,8,3
3,844,OPERATIONAL,251 Store ET Gerji store,ChIJ-60ObDCFSxYR8qG94H3GuPg,-,9.001777,38.810458,0,0,0,...,11,1,6,0,13,153,27,0,35,19
4,1045,OPERATIONAL,Selam Super Market,ChIJ-7iRqQ6FSxYRDfOlAYW38c8,-,9.011199,38.796551,0,0,0,...,22,6,18,1,9,110,21,0,16,24


In [31]:
from math import sin, cos, sqrt, atan2, radians

def calculateDistance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1, lon1 = radians(lat1), radians(lon1)
    lat2, lon2 = radians(lat2), radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [32]:
def collectAndFindNearbyPlaces(row, radius):
    lat, lon = row.latitude, row.longitude
    counter = {}
    for key in nearby_places_csvs:
        counter[key] = 0
    for i in nearby_places.itertuples(index=True, name='Pandas'):
        if calculateDistance(lat, lon, i.latitude, i.longitude)<=radius:
            counter[i.type] += 1
    return pd.Series(list(counter.values()) ,index=list(counter))

In [33]:
#1km radius
supermarkets[list(nearby_places_csvs)] = supermarkets.apply(collectAndFindNearbyPlaces, radius=1, axis=1)

In [35]:
supermarkets.describe()

,Unnamed: 0,latitude,longitude,rv_park,beauty_salon,aquarium,home_goods_store,storage,night_club,bicycle_store,...,churches,gas_stations,hospitals,mosques,pharmacies,restaurants,schools,train_stations,competitors_supermarket,competitors_shopping_mall
count,1392.000000,1392.000000,1392.000000,1392.0,1392.0,1392.0,1392.0,1392.0,1392.0,1392.0,...,1392.000000,1392.000000,1392.000000,1392.000000,1392.000000,1392.000000,1392.000000,1392.000000,1392.000000,1392.000000
mean,1139.927443,9.003146,38.765421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.777299,2.802443,9.524425,4.133621,8.293822,86.214799,21.183190,0.033046,12.211925,20.731322
std,686.825950,0.034234,0.046179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.548595,2.543747,9.043674,3.967259,5.750306,61.134592,13.380197,0.178821,7.513730,18.247273
min,0.000000,8.835770,38.662204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,563.500000,8.988069,38.736376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.000000,1.000000,3.000000,1.000000,4.000000,32.000000,13.000000,0.000000,7.000000,6.000000
50%,1121.000000,9.005729,38.759672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.000000,2.000000,7.000000,3.000000,8.000000,73.000000,20.000000,0.000000,12.000000,15.000000
75%,1668.250000,9.024217,38.788254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.000000,5.000000,14.000000,5.000000,11.000000,133.000000,26.000000,0.000000,17.000000,32.000000
max,2553.000000,9.102839,38.920221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.000000,11.000000,45.000000,20.000000,26.000000,257.000000,79.000000,1.000000,35.000000,72.000000


In [12]:
supermarkets.drop_duplicates(inplace = True, subset=['name', 'latitude', 'longitude'])

In [13]:
supermarkets.shape

(1388, 16)

In [36]:
import json
import os

script_dir = os.path.dirname("__file__")
subcity_population_json = os.path.join(script_dir, '../Data/Population_per_subcity.json')
subcity_population = open(subcity_population_json,)
subcity_population_data = json.load(subcity_population)

In [38]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import numpy as np

def determineSubcityAndAddPopulation(row):
    lat, lon = row.latitude, row.longitude
    for subcity in subcity_population_data:
        point = Point(lat,lon)
        polygon = Polygon([(i,j) for i, j in subcity_population_data[subcity]['coordinates']])
        if polygon.contains(point):
            total_children = total_working = total_elderly = 0
            total_males = total_females = 0
            for age in subcity_population_data[subcity]['population']:
                if age=="0 - 4" or age=="5 - 9" or age=="10-14":
                    total_children +=subcity_population_data[subcity]['population'][age]['Total']
                elif age=="65-69" or age=="70-74" or age=="75-79" or age=="80-84" or age=='85-89' or age=="90-94" or age=="95+":
                    total_elderly +=subcity_population_data[subcity]['population'][age]['Total'] 
                else:
                    total_working +=subcity_population_data[subcity]['population'][age]['Total'] 
                total_males +=subcity_population_data[subcity]['population'][age]['Males'] 
                total_females +=subcity_population_data[subcity]['population'][age]['Females']
            return pd.Series([total_males, total_females, total_children, total_working, total_elderly] ,index=['Males', 'Females', 'Children', 'Working', 'Elderly']) 
    return pd.Series([0,0,0,0,0] ,index=['Males', 'Females', 'Children', 'Working', 'Elderly']) 




In [16]:
supermarkets['Males'] = 0
supermarkets['Females'] = 0
supermarkets['Children'] = 0
supermarkets['Working'] = 0
supermarkets['Elderly'] = 0

In [39]:
supermarkets[['Males', 'Females', 'Children', 'Working', 'Elderly']] = supermarkets.apply(determineSubcityAndAddPopulation, axis=1)

In [40]:
supermarkets.head()

,Unnamed: 0,business_status,name,place_id,rating,latitude,longitude,rv_park,beauty_salon,aquarium,...,restaurants,schools,train_stations,competitors_supermarket,competitors_shopping_mall,Males,Females,Children,Working,Elderly
0,1472,OPERATIONAL,"AllMart Plus, Ayat Neighborhood, Addis Ababa",ChIJ-3rbDsqRSxYRKX0Zr55zXwY,-,9.020171,38.875108,0,0,0,...,28,4,0,4,6,145225,163770,75491,225517,7987
1,1361,OPERATIONAL,Nahuta SuperMarket,ChIJ-4OScHqFSxYRPmsu7L3m4LA,5.0,9.022502,38.785852,0,0,0,...,105,28,0,19,21,161592,185072,83403,251219,12042
2,1796,OPERATIONAL,Afran Mini Market,ChIJ-5L2YgKJSxYRj__fg6gaEy0,-,9.057363,38.724341,0,0,0,...,54,20,0,8,3,129396,138228,61641,195556,10427
3,844,OPERATIONAL,251 Store ET Gerji store,ChIJ-60ObDCFSxYR8qG94H3GuPg,-,9.001777,38.810458,0,0,0,...,153,27,0,35,19,145225,163770,75491,225517,7987
4,1045,OPERATIONAL,Selam Super Market,ChIJ-7iRqQ6FSxYRDfOlAYW38c8,-,9.011199,38.796551,0,0,0,...,110,21,0,16,24,145225,163770,75491,225517,7987


In [41]:
supermarkets.to_csv('supermarkets_cleaned_with_popn_v3.csv')